In [1]:
!pip install streamlit faiss-cpu sentence-transformers groq python-docx PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os

# 🔐 Store your Groq API key securely
os.environ["GROQ_API_KEY"] = " "


In [8]:
import os
import streamlit as st
from PyPDF2 import PdfReader
from docx import Document
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq

# ✅ Get Groq API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
INDEX = faiss.IndexFlatL2(384)
stored_chunks = []

# UI Styling
st.markdown("""
    <style>
    .main-title {
        font-size: 40px;
        color: #2E86C1;
        font-weight: bold;
        text-align: center;
        margin-bottom: 30px;
    }
    .card {
        background-color: #ffffff;
        padding: 20px;
        border-radius: 15px;
        box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
        margin-top: 20px;
    }
    body {
        background-color: #f8fbfd;
    }
    </style>
""", unsafe_allow_html=True)

st.markdown('<div class="main-title">📄 Smart RAG Document QA Assistant</div>', unsafe_allow_html=True)

def extract_text(file):
    if file.type == "application/pdf":
        reader = PdfReader(file)
        return " ".join([page.extract_text() or "" for page in reader.pages])
    elif file.type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        doc = Document(file)
        return "\n".join([p.text for p in doc.paragraphs])
    elif file.type.startswith("text"):
        return file.read().decode("utf-8")
    return ""

def chunk_text(text, chunk_size=200):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

def store_embeddings(chunks):
    vectors = embed_model.encode(chunks)
    INDEX.add(np.array(vectors, dtype=np.float32))
    stored_chunks.extend(chunks)

def retrieve_similar_chunks(query, top_k=3):
    query_vector = embed_model.encode([query])
    distances, indices = INDEX.search(np.array(query_vector, dtype=np.float32), top_k)
    return [stored_chunks[i] for i in indices[0]]

def get_llm_answer(query, context):
    prompt = f"Answer the question based on the following context:\n\n{context}\n\nQuestion: {query}"
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-70b-8192"
    )
    return chat_completion.choices[0].message.content

uploaded_file = st.file_uploader("📁 Upload your document", type=["pdf", "docx", "txt"])
query = st.text_input("💬 Ask a question about your document")

if uploaded_file:
    with st.spinner("Processing file..."):
        text = extract_text(uploaded_file)
        chunks = chunk_text(text)
        store_embeddings(chunks)
    st.success("✅ Document uploaded and indexed!")

if st.button("🧠 Get Answer") and query:
    with st.spinner("Thinking..."):
        context = "\n\n".join(retrieve_similar_chunks(query))
        answer = get_llm_answer(query, context)
        st.markdown(f'<div class="card"><b>Answer:</b><br>{answer}</div>', unsafe_allow_html=True)

st.markdown("<br><center style='color: grey;'>Built by Muqadas with ❤️ using Streamlit + Groq + FAISS</center>", unsafe_allow_html=True)


2025-07-16 09:53:57.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.361 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-16 09:53:57.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [9]:
!wget -q -O - ipv4.icanhazip.com

34.23.184.103


In [ ]:

# Step 4: Run the Streamlit app with LocalTunnel on port 8501
!streamlit run app.py & npx localtunnel --port 8501


⠙

⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.23.184.103:8501

⠦⠧⠇⠏⠋⠙your url is: https://cruel-swans-tell.loca.lt
2025-07-16 09:55:18.895417: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752659718.930478    5577 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752659718.939944    5577 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:19005 (check your firewall settings)
    at Socket.<anonymous> (/root